In [127]:
import numpy as np
import matplotlib.pyplot as plt
from procesar_ecuacion import procesar_ecuacion

### Matriz bonita

In [128]:
# Función para imprimir unha matriz bonita
def print_matriz(m, nome=None, despois=None):
    n_filas = len(m)
    n_columnas = len(m[0])

    if nome is not None:
        print("{}".format(nome))

    anchuras = [max([len(str(m[i][j])) for i in range(n_filas)]) for j in range(n_columnas)]
    for i in range(len(m)):
        coefs = " ".join([str(m[i][j]).rjust(max(anchuras[:-1])) for j in range(n_columnas-1)])
        print(coefs, "|", str(m[i][-1]).rjust(anchuras[-1]))
        
    if despois is not None:
        print(despois)

### Convertir ecuacións a numpy

Exemplo sobre como procesar unha lista de ecuacións en formato textual a un array de numpy utilizando o módulo `procesar_ecuacions.py`.

In [129]:
ecuacions = (
    "3x - 2y + 4z = 0",
    "5x + 3y = 3",
)

print_matriz(procesar_ecuacion(*ecuacions))

 3 -2  4 | 0
 5  3  0 | 3


### Manipular as matrices do sistema

In [130]:
# Obter as matrices de coeficientes e o vector de termos independientes de un sistema conxunto
def dividir_sistema(ecuacions):
    coefs = np.array([x[:-1] for x in ecuacions])
    t_ind = np.array([x[-1] for x in ecuacions])
    return coefs, t_ind

# Inversa da función anterior, devolve un sistema combinado
def combinar_sistema(coefs, t_ind):
    return np.array([[x for x in coefs[i]] + [t_ind[i]] for i in range(len(coefs))])

### Triangulación de matrices

Eliminación de Gauss

In [131]:
# Eliminación de Gauss
def eliminacion_gauss(sistema, columna, salida=True):
    p = sistema[eliminacion_gauss.fila_pivote][columna]

    eliminacion_gauss.i += 1
    if salida:
        print("Columna {}, Fila pivote: {}".format(eliminacion_gauss.i, eliminacion_gauss.fila_pivote+1))

    for f in eliminacion_gauss.outras_filas:
        # Cálculo coeficiente
        k = sistema[f][columna] / p
        if salida:
            print("Fila: {}, a_{}{} / a_{}{} = {}".format(columna+1, f+1, f+1, columna+1, eliminacion_gauss.fila_pivote+1, columna+1, k))
        
        # Restar a N final menos o producto do coeficiente pola fila pivote
        temp = np.copy(sistema[f])
        sistema[f] -= k * sistema[eliminacion_gauss.fila_pivote]
        if salida:
            stringify = lambda arr : " ".join([str(x) for x in arr[:-1]]) + " | " + str(arr[-1])
            print("{}  >>  {}".format(stringify(temp), stringify(sistema[f])))
    
    for f in eliminacion_gauss.outras_filas:
        if sistema[f][columna] == 0:
            eliminacion_gauss.fila_pivote = f
            eliminacion_gauss.outras_filas.remove(f)
            break

    if salida:
        print_matriz(sistema, "Sistema:", "")

# Transforma unha matriz nunha matriz triangular superior
def sin_pivote(ecuacions, fila_pivote=0, salida=True):
    sistema = ecuacions.astype(float) # Sistema de ecuacions sobre o que operar
    
    # Parametros iniciais da eliminación de Gauss
    eliminacion_gauss.i = 0
    eliminacion_gauss.fila_pivote = fila_pivote
    eliminacion_gauss.outras_filas = [f for f in range(len(sistema)) if f != fila_pivote] # Filas distintas da pivote
    
    # Iterar polos elementos do triángulo iferior da matriz
    for c in range(len(ecuacions[0])):
        eliminacion_gauss(sistema, c, salida)
        if len(eliminacion_gauss.outras_filas) == 0:
            break

    return sistema


### Cálculo de soluciones

Sustitución regresiva

In [132]:
def comprobar_matriz_triangular(m):
    for i in range(len(m)):
        for j in range(len(m[0])):
            if i > j and m[i][j] != 0:
                return False
    return True

def ordear_matriz_triangular(m):
    if not comprobar_matriz_triangular(m):
        raise ValueError("A matriz non é triangular superior")
    m_ = np.copy(m).tolist()
    m_.sort(key = lambda m: m[:-1].count(0))
    return np.array(m_)

In [133]:
def sustitucion_regresiva(sistema_reducido):
    m = ordear_matriz_triangular(sistema_reducido)
    solucions = np.zeros(len(m))

    for i in range(len(m))[::-1]:
        a = m[i][:-1] * solucions
        solucions[i] = (m[i][-1] - sum(a)) / m[i][i]

    return solucions

In [134]:
solucions_a_texto = lambda sol, keys: " , ".join(["{}: {}".format(list(keys)[i], sol[i]) for i in range(len(sol))])

## Exercicio 1

In [135]:
# Sistema de ecuacións plantexado

ecuacions_s = (
    "2x1 -  x2 + x3  = 3",
    "-x1 +  x2 + 2x3 = 7",
    " x1 + 2x2 - x3  = 2",
)

ec, keys = procesar_ecuacion(*ecuacions_s, nomes=True)
print_matriz(ec, "Sistema inicial:", "")

sistema_reducido = sin_pivote(ec)
print_matriz(sistema_reducido, "Sistema final:", "")

sol = sustitucion_regresiva(sistema_reducido)
print("Solucións:", solucions_a_texto(sol, keys))

Sistema inicial:
 2 -1  1 | 3
-1  1  2 | 7
 1  2 -1 | 2

Columna 1, Fila pivote: 1
Fila: 1, a_22 / a_11 = 1
-1.0 1.0 2.0 | 7.0  >>  0.0 0.5 2.5 | 8.5
Fila: 1, a_33 / a_11 = 1
1.0 2.0 -1.0 | 2.0  >>  0.0 2.5 -1.5 | 0.5
Sistema:
 2.0 -1.0  1.0 | 3.0
 0.0  0.5  2.5 | 8.5
 0.0  2.5 -1.5 | 0.5

Columna 2, Fila pivote: 2
Fila: 2, a_33 / a_22 = 2
0.0 2.5 -1.5 | 0.5  >>  0.0 0.0 -14.0 | -42.0
Sistema:
  2.0  -1.0   1.0 |   3.0
  0.0   0.5   2.5 |   8.5
  0.0   0.0 -14.0 | -42.0

Sistema final:
  2.0  -1.0   1.0 |   3.0
  0.0   0.5   2.5 |   8.5
  0.0   0.0 -14.0 | -42.0

Solucións: x1: 1.0 , x2: 2.0 , x3: 3.0


### Proba con sistemas de orden superior

In [136]:
# Sistema de ecuacións plantexado

ecuacions_s2 = (
    "2x1 -  x2 + x3  - 2x4 = 3",
    "-x1 +  x2 + 2x3 - 3x4 = 7",
    " x1 + 2x2 - x3  +  x4 = 2",
    "3x1 + 2x2 - x3  - 5x4 = 9",
)

ec_2, keys_2 = procesar_ecuacion(*ecuacions_s2, nomes=True)
print_matriz(ec_2, "Sistema inicial:", "")

sistema_reducido_2 = sin_pivote(ec_2, salida=False)
print_matriz(np.round(sistema_reducido_2, 2), "Sistema final:", "")

sol_2 = np.round(sustitucion_regresiva(sistema_reducido_2), 2)
print("Solucións:", solucions_a_texto(sol_2, keys_2))

Sistema inicial:
 2 -1  1 -2 | 3
-1  1  2 -3 | 7
 1  2 -1  1 | 2
 3  2 -1 -5 | 9

Sistema final:
  2.0  -1.0   1.0  -2.0 |   3.0
  0.0   0.5   2.5  -4.0 |   8.5
  0.0   0.0 -14.0  22.0 | -42.0
  0.0   0.0   0.0 -5.43 |   5.0

Solucións: x1: 0.74 , x2: 1.87 , x3: 1.55 , x4: -0.92


### Pivoteo parcial

In [137]:
# Obter a columna dun array 2D
columna = lambda a, n: a[:, n]

def pivoteo_parcial(ecuacions, fila_pivote=0, salida=True):
    sistema = ecuacions.astype(float) # Sistema de ecuacions sobre o que operar
    n_columnas = len(sistema[0]) # Número de columnas

    # Parametros iniciais da eliminación de Gauss
    eliminacion_gauss.i = 0
    eliminacion_gauss.fila_pivote = fila_pivote
    eliminacion_gauss.outras_filas = [f for f in range(len(sistema)) if f != fila_pivote] # Filas distintas da pivote

    for i in range(n_columnas - 1):
        c = columna(sistema, i)

        # Pivotar fila
        fila_max = abs(c[i:]).argmax() + i
        if fila_max != i:
            sistema[[i, fila_max]] = sistema[[fila_max, i]]
        
        # Eliminación Gauss
        eliminacion_gauss(sistema, i, salida)
        if len(eliminacion_gauss.outras_filas) == 0:
            break
        
    return sistema

## Exercicio 3

In [138]:
ecuacions_pv = (
    " x1 + 2x2 -  x3 = 2",
    "2x1 + 4x2 + 5x3 = 25",
    "3x1 -  x2 - 2x3 = -5",
)

ec_pv, keys_pv = procesar_ecuacion(*ecuacions_pv, nomes=True)
print_matriz(ec_pv, "Sistema inicial:", "")

sr_pv = pivoteo_parcial(ec_pv, salida=False)
print_matriz(np.round(sr_pv, 2), "Sistema final:", "")

sol_pv = sustitucion_regresiva(sr_pv)
print("Solucións:", solucions_a_texto(np.round(sol_pv, 2), keys_pv))

Sistema inicial:
 1  2 -1 |  2
 2  4  5 | 25
 3 -1 -2 | -5

Sistema final:
 3.0 -1.0 -2.0 |  -5.0
 0.0 4.67 6.33 | 28.33
 0.0  0.0 -3.5 | -10.5

Solucións: x1: 1.0 , x2: 2.0 , x3: 3.0


## Exercicio 4

In [139]:
ecuacions_pv = (
    "x1 + 2x2 - 3x3 - x4 = -2.5",
    "-3x2 + 2x3 + 6x4 = 7",
    "-3x1 - x2 + 3x3 + x4 = 2.5",
    "2x1 + 3x2 + 2x3 - x4 = -10.5",
)

ec_pv, keys_pv = procesar_ecuacion(*ecuacions_pv, nomes=True)
print_matriz(ec_pv, "Sistema inicial:", "")

sr_pv = pivoteo_parcial(ec_pv, salida=False)
print_matriz(np.round(sr_pv, 2), "Sistema final:", "")

sol_pv = sustitucion_regresiva(sr_pv)
print("Solucións:", solucions_a_texto(np.round(sol_pv, 2), keys_pv))

Sistema inicial:
 1.0  2.0 -3.0 -1.0 |  -2.5
 0.0 -3.0  2.0  6.0 |   7.0
-3.0 -1.0  3.0  1.0 |   2.5
 2.0  3.0  2.0 -1.0 | -10.5

Sistema final:
-3.0 -1.0  3.0  1.0 |   2.5
 0.0 -3.0  2.0  6.0 |   7.0
 0.0  0.0 5.56 4.33 | -3.39
 0.0  0.0  0.0 3.36 |  1.68

Solucións: x1: -1.0 , x2: -2.0 , x3: -1.0 , x4: 0.5


### Proba cun sistema de 6 ecuacións

In [140]:
ecuacions_pv = (
    "2x1 -  x2 + x3  - 2x4 + 4x5 = 3",
    "-x1 +  x2 + 2x3 - 3x4 + 8x5 = 7",
    "      2x2       +  x4 + 9x5 = 2",
    "3x1 + 2x2 - x3  - 5x4       = 9",
    "9x1 + 4x2 - 8x3 - 3x4 + 4x5 = 9",
)

ec_pv, keys_pv = procesar_ecuacion(*ecuacions_pv, nomes=True)
print_matriz(ec_pv, "Sistema inicial:", "")

sr_pv = pivoteo_parcial(ec_pv, salida=False)
print_matriz(np.round(sr_pv, 2), "Sistema final:", "")

sol_pv = sustitucion_regresiva(sr_pv)
print("Solucións:", solucions_a_texto(np.round(sol_pv, 2), keys_pv))


Sistema inicial:
 2 -1  1 -2  4 | 3
-1  1  2 -3  8 | 7
 0  2  0  1  9 | 2
 3  2 -1 -5  0 | 9
 9  4 -8 -3  4 | 9

Sistema final:
  9.0   4.0  -8.0  -3.0   4.0 |  9.0
  0.0   2.0   0.0   1.0   9.0 |  2.0
  0.0   0.0  2.78 -0.39 11.61 | 2.89
  0.0   0.0   0.0  -4.1 -11.3 |  3.6
  0.0   0.0   0.0   0.0  8.05 | 1.98

Solucións: x1: -0.11 , x2: 0.67 , x3: -0.2 , x4: -1.55 , x5: 0.25
